# Study the effect of spacecraft spin on proton noise

###### below we attempt to directly integrate the triple integral given in Issautier et al (1999). But it seems not feasible; the inner two integral already took too much time. 

In [1]:
import sys
sys.path.append("..")
import numpy as np
import scipy.integrate
from qtn.util import zpd_sp, timing, zp_pade
import matplotlib.pyplot as plt

In [2]:
boltzmann = 1.3806488e-23  # J/K
emass = 9.10938291e-31     # kg
pmass = 1.67262178e-27
echarge = 1.60217657e-19   # C
permittivity = 8.854187817e-12  # F/m
cspeed = 299792458         # m/s

In [3]:
def cos_gamma(u, beta, phi):
    """
    cos_gamma = u * cos(beta) + sqrt(1-u^2) * sin(beta) * cos(phi)
    
    Key parameters
    --------------
    u: cos(theta), where theta is the angle between wavenumber vector and solar wind bulk flow.
    beta: angle between wire antenna and solar wind bulk flow (\vec{v_{sw}}).
    phi: azimuthal angle in the plane perpendicular to \vec{v_{sw}}
    
    Return
    ------
    return cos(gamma)
    """
    return u * np.cos(beta) + np.sqrt(1-u**2) * np.sin(beta) * np.cos(phi)

In [4]:
def phi_integrand(k, l, u, beta, phi):
    """
    sin(k*l*cos_gamma/2)**4 / (k*l*cos_gamma)**2
    
    Key parameters
    --------------
    k: magnitude of wavenumber vector
    l: antenna length (monopole)
    u, beta, phi: defined in cos_gamma() function
    
    Return:
    return the integrand of phi integral
    """
    cg = cos_gamma(u, beta, phi)
    return np.sin(k * l * cg/2) **4 / (k * l * cg)**2

In [5]:
def phi_integral(k, l, u, beta):
    """
    Integrate phi_integrand over (0, 2pi)
    """
    return scipy.integrate.quad(lambda phi: phi_integrand(k, l, u, beta, phi), 0, 2*np.pi)[0]

In [6]:
def e_l(k, w, vsw, u, tp, n):
    """
    Calculate the longitudinal susceptibility tensor.
    
    Key parameters
    --------------
    k: magnitude of the wavenumber vector
    w: frequency of interest
    vsw: solar wind speed
    u: cos(theta) which is defined in cos_gamma()
    tp: proton temperature
    np: proton density
    
    Return
    ------
    Return the longitudinal susceptibility tensor.
    
    """
    vthp = np.sqrt(2 * boltzmann * tp/ pmass)
    zetap = (w - k*vsw*u)/ (k * vthp)
    #print(zetap)
    prod_1 = (emass/k)**2 / permittivity * n/(boltzmann * tp)
    prod_2 = zpd_sp(zetap)
    return 1 + prod_1 * prod_2

In [7]:
def u_integrand(k, w, l, vsw, u, tp, n, beta):
    """
    Calculate the integrand of u integral.
    """
    el = e_l(k, w, vsw, u, tp, n)
    vthp = np.sqrt(2 * boltzmann * tp/ pmass) 
    zetap = (w - k*vsw*u)/ (k * vthp)
    return np.exp(-zetap**2) * phi_integral(k, l, u, beta) / np.abs(el)**2
    

In [8]:
@timing
def u_integral(k, w, l, vsw, tp, n, beta):
    """
    Calculate the u integral
    
    """
    return scipy.integrate.quad(lambda u: u_integrand(k, w, l, vsw, u, tp, n, beta), -1, 1)[0]

In [9]:
u_integral(10, 1e4, 50, 4e5, 1e5, 1e7, 0.5)

/tmp/ipykernel_3902248/2291627588.py:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return scipy.integrate.quad(lambda phi: phi_integrand(k, l, u, beta, phi), 0, 2*np.pi)[0]


u_integral function took 30454.486 ms


/tmp/ipykernel_3902248/1485099015.py:7: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return scipy.integrate.quad(lambda u: u_integrand(k, w, l, vsw, u, tp, n, beta), -1, 1)[0]


0.0011936989750032973

In [10]:
u_integral(3, 1e4, 50, 4e5, 1e5, 1e7, 0.5)

u_integral function took 4863.371 ms


0.003979874789559081

In [11]:
ulist = np.linspace(-1, 1, 1001)
uintval = [u_integrand(50, 1e4, 50, 4e5, u, 1e5, 1e7, 0.5) for u in ulist]
ellist = [e_l(50, 1e4, 4e5, u, 1e5, 1e7) for u in ulist]

/tmp/ipykernel_3902248/2291627588.py:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return scipy.integrate.quad(lambda phi: phi_integrand(k, l, u, beta, phi), 0, 2*np.pi)[0]
/tmp/ipykernel_3902248/2291627588.py:5: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return scipy.integrate.quad(lambda phi: phi_integrand(k, l, u, beta, phi), 0, 2*np.pi)[0]


### Conclusion

###### It takes almost forever to do the full three dimension integral. Some simplications is necessary. 

## Simplified expression for angle dependent proton noise

In [12]:
import numpy as np
import scipy.integrate
from qtn.util import timing, boltzmann, emass, echarge, permittivity, cspeed
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def phi_integrand(y, beta, phi, omega, lrel):
    """
    integrand for the phi integral.
    
    Key parameters
    --------------
    y: dimensionless parameter for the outer integral
    beta: angle between antenna and solar wind velocity
    phi: azimuthal angle
    lrel: antenna length/debye length
    
    Return
    ------
    return the integrand for the phi integral
    """
    kl_cos_gamma = lrel * (omega * np.cos(beta) + y * np.sin(beta) * np.cos(phi))
    return np.sin(0.5 * kl_cos_gamma)**4 / kl_cos_gamma**2

def phi_integral(y, beta, omega, lrel):
    """
    value of the phi integral
    
    """
    return scipy.integrate.quad(lambda phi: phi_integrand(y, beta, phi, omega, lrel), 0, 2*np.pi)[0]

In [14]:
@timing
def proton_angle(beta, f, ne, n, t, tp, tc, k, vsw, ant_len):
    """
    proton noise at an arbitrary angle between antenna and solar wind velocity
    
    Key parameters
    --------------
    
    """
    ne = ne * 1e6
    tp = tp * echarge/boltzmann
    tc = tc * echarge/boltzmann
    w_p = np.sqrt(echarge**2 * ne/emass/permittivity)
    te = (tc + tc * t * n)/(1+n)
    tg = tc * (1 + n)/(1 + n/t)
    ld = np.sqrt(permittivity * boltzmann * tg/ne/ echarge**2)
    lrel = ant_len/ld
    vte = np.sqrt(2 * boltzmann * te/ emass)
    omega = f * 2 * np.pi * ld/vsw
    tep = tg/tp
    M = vsw/vte
    
    integrand = lambda y: y * phi_integral(y, beta, omega, lrel) / (1 + y**2 + omega**2) / (1 + tep + y**2 + omega**2)
    integral = scipy.integrate.quad(integrand, 0, np.inf)[0]
    coeff = 4 * np.sqrt(2 * emass * boltzmann * tg)/(np.pi**2 * permittivity * M)
    return coeff * integral

In [15]:
ant_len = 50      # m (monopole) 
ant_rad = 1.9e-4  # m
base_cap = 20e-12 # Fara
fbins = np.array([4000*2**((2*i+1)/32) for i in range(96)])

ne = 12.28
vsw=3.966e5
tc = 9.91
tp = 9.62
t = 6.57
n = 0.026
fpe = 31.47e3
k = 7

### perpendicular case

###### (can also be calculated using proton_angle routine, but much faster using MaxKappa.proton routine)

In [16]:
from qtn.maxkappa import MaxKappa

In [17]:
p = MaxKappa(ant_len, ant_rad, base_cap)

In [18]:
band_a = fbins[0:32]

In [19]:
perp = np.array([p.proton(f, ne, n, t, tp, tc, k, vsw) for f in fbins])

### other angles ($60^\circ$, $45^\circ$ and $30^\circ$)

In [20]:
sixty = np.array([proton_angle(np.pi/3, f, ne, n, t, tp, tc, k, vsw, ant_len) for f in fbins])

/tmp/ipykernel_3902248/1544233198.py:24: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return scipy.integrate.quad(lambda phi: phi_integrand(y, beta, phi, omega, lrel), 0, 2*np.pi)[0]
/tmp/ipykernel_3902248/1544233198.py:24: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return scipy.integrate.quad(lambda phi: phi_integrand(y, beta, phi, omega, lrel), 0, 2*np.pi)[0]


proton_angle function took 3184.724 ms
proton_angle function took 3912.300 ms
proton_angle function took 5089.588 ms
proton_angle function took 3973.263 ms
proton_angle function took 4015.414 ms
proton_angle function took 3531.065 ms
proton_angle function took 4254.112 ms
proton_angle function took 3694.366 ms
proton_angle function took 2161.685 ms
proton_angle function took 1673.270 ms
proton_angle function took 2259.661 ms
proton_angle function took 8075.268 ms
proton_angle function took 10054.128 ms
proton_angle function took 6291.162 ms
proton_angle function took 6219.735 ms
proton_angle function took 862.096 ms
proton_angle function took 6153.745 ms
proton_angle function took 7629.044 ms
proton_angle function took 6316.703 ms
proton_angle function took 7780.454 ms
proton_angle function took 6087.740 ms
proton_angle function took 12125.928 ms
proton_angle function took 6291.732 ms
proton_angle function took 6506.039 ms
proton_angle function took 5616.034 ms
proton_angle function to

In [21]:
forty_five = np.array([proton_angle(np.pi/4, f, ne, n, t, tp, tc, k, vsw, ant_len) for f in fbins])
thirty = np.array([proton_angle(np.pi/6, f, ne, n, t, tp, tc, k, vsw, ant_len) for f in fbins])
fifteen = np.array([proton_angle(np.pi/12, f, ne, n, t, tp, tc, k, vsw, ant_len) for f in fbins])

/tmp/ipykernel_3902248/1544233198.py:24: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return scipy.integrate.quad(lambda phi: phi_integrand(y, beta, phi, omega, lrel), 0, 2*np.pi)[0]
/tmp/ipykernel_3902248/1544233198.py:24: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return scipy.integrate.quad(lambda phi: phi_integrand(y, beta, phi, omega, lrel), 0, 2*np.pi)[0]


proton_angle function took 3181.652 ms
proton_angle function took 3061.968 ms
proton_angle function took 1524.245 ms
proton_angle function took 3776.998 ms
proton_angle function took 3198.457 ms
proton_angle function took 3164.607 ms
proton_angle function took 3658.770 ms
proton_angle function took 3886.213 ms
proton_angle function took 1740.637 ms
proton_angle function took 3723.236 ms
proton_angle function took 4481.536 ms
proton_angle function took 3918.690 ms
proton_angle function took 4512.982 ms
proton_angle function took 3069.242 ms
proton_angle function took 1806.803 ms
proton_angle function took 1734.136 ms
proton_angle function took 3527.515 ms
proton_angle function took 1916.060 ms
proton_angle function took 5392.110 ms
proton_angle function took 3847.247 ms
proton_angle function took 1712.390 ms
proton_angle function took 1835.495 ms
proton_angle function took 1901.527 ms
proton_angle function took 1905.389 ms
proton_angle function took 1818.804 ms
proton_angle function too

proton_angle function took 3014.846 ms
proton_angle function took 2848.612 ms
proton_angle function took 2645.995 ms
proton_angle function took 2208.574 ms
proton_angle function took 2442.443 ms
proton_angle function took 2494.035 ms
proton_angle function took 3060.412 ms
proton_angle function took 3312.365 ms
proton_angle function took 4506.650 ms
proton_angle function took 4634.728 ms
proton_angle function took 3639.558 ms
proton_angle function took 2065.373 ms
proton_angle function took 3916.068 ms
proton_angle function took 3576.833 ms
proton_angle function took 2579.685 ms
proton_angle function took 2931.144 ms
proton_angle function took 2860.111 ms
proton_angle function took 3085.522 ms
proton_angle function took 3047.016 ms
proton_angle function took 4750.588 ms
proton_angle function took 3595.011 ms
proton_angle function took 1993.574 ms
proton_angle function took 2659.689 ms
proton_angle function took 2782.907 ms
proton_angle function took 2525.048 ms
proton_angle function too

###### save proton noise to file

In [22]:
#np.savez('625_proton_angle', perp = perp, sixty = sixty,
#         forty_five = forty_five, thirty = thirty, fifteen = fifteen)

In [23]:
test=np.load('625_proton_angle.npz')
test.files

FileNotFoundError: [Errno 2] No such file or directory: '625_proton_angle.npz'

In [ ]:
saved_perp = test['perp']
saved_sixty = test['sixty']
saved_forty_five = test['forty_five']
saved_thirty = test['thirty']
saved_fifteen = test['fifteen']

In [ ]:
from scipy.io.idl import readsav
from os.path import expanduser
home = expanduser("~")
fbins = np.array([4000*2**((2*i+1)/32) for i in range(96)])

#### plot proton noise at different beta angles

In [ ]:
fig = plt.figure(figsize=[6,6])
plt.plot(fbins, saved_perp, label='$(\mathbf{v}_{sw}, \hat{\mathbf{l}}_{ant})=90^\circ$')
plt.plot(fbins, saved_sixty, label='$60^\circ$')
plt.plot(fbins, saved_forty_five, label='$45^\circ$')
plt.plot(fbins, saved_thirty, label='$30^\circ$')
plt.plot(fbins, saved_fifteen, label='$15^\circ$')

plt.xscale('log')
plt.yscale('log')
plt.xlim([4e3, 256e3])
plt.ylim([1e-18, 1e-12])
plt.legend(loc='best')
plt.show()

In [ ]:
m_dat=readsav(home+'/Google Drive/research/data/meudon_tnr/TNR_XY_ACE_19950625.sav')
m_data=m_dat['data']
mt = m_data['timeur8'][0]
mtag = m_data['time'][0]
spec = m_data['spectra'][0]#[:, 5223]
spec = 10. ** (spec/10.)

In [ ]:
spec[:, 5223:5225].shape

In [ ]:
dat625 = np.load('data/625_bimax.npz')
gain = dat625['gain']
e_noise = dat625['e_noise']
s_noise = dat625['s_noise']
p_noise = dat625['p_noise']

In [ ]:
dat625.files

In [ ]:
np.arange(5220, 5228)

In [ ]:
ind = 5223
rang = 3
avg_spec = np.average(spec[:, ind-rang:ind + rang + 1], axis=1)

fig = plt.figure(figsize=[6,6])
plt.rc('text', usetex=False)
plt.rc('font', family='serif')

plt.plot(fbins, saved_perp + e_noise, label='$90^\circ$')
plt.plot(fbins, saved_sixty + e_noise, label='$60^\circ$')
plt.plot(fbins, saved_forty_five + e_noise, label='$45^\circ$')
plt.plot(fbins, saved_thirty + e_noise, label='$30^\circ$')
plt.plot(fbins, saved_fifteen +e_noise, label='$15^\circ$')

for i in np.arange(ind-rang, ind+rang+1):
    plt.plot(fbins, spec[:, i] * gain, 'o', markersize=1)
plt.plot(fbins, avg_spec * gain, '-', markersize=1, label='average TNR')

plt.xscale('log')
plt.yscale('log')
plt.xlim([4e3, 256e3])
plt.ylim([1e-16, 1e-12])
plt.title("theoretical spectra for " + mtag[ind].decode("utf-8") + "\n dots correspond to 7 nearby TNR spectra")
plt.legend(loc='best')
plt.show()

In [ ]:
fig.savefig('wind1.png', dpi=300)

In [ ]:
beta_list = np.arange(-np.pi, np.pi+0.01, np.pi/6)

In [ ]:
p_beta = np.array([proton_angle(beta, 4e3, ne, n, t, tp, tc, k, vsw, ant_len) for beta in beta_list])

In [ ]:
plt.plot(beta_list, p_beta/gain[0])
plt.yscale('log')
plt.show()

In [ ]:
proton_angle(1.57, 14e3, ne, n, t, tp, tc, k, vsw, ant_len)